In [5]:
df = pd.read_csv('numerai_training_data.csv')
df.sample(5)

data = {'X': df.drop('target', axis=1), 'y': df['target']}

In [56]:
import tensorflow as tf

n_samples = data['X'].shape[0]
n_features = data['X'].shape[1]
n_hidden = 10


X = tf.placeholder(dtype=tf.float32, shape=[None, n_features])
y = tf.placeholder(dtype=tf.float32, shape=[None, 1])


init = pd.Series()
init['W1'] = tf.random_normal(seed=13, shape=[n_features, n_hidden], dtype=tf.float32, mean=0.0, stddev=np.sqrt(6.0 / n_features))
init['b1'] = tf.random_normal(seed=13, shape=[n_hidden], dtype=tf.float32, mean=0.0, stddev=np.sqrt(6.0))
init['W2'] = tf.random_normal(seed=13, shape=[n_hidden, 1], dtype=tf.float32, mean=0.0, stddev=np.sqrt(6.0 / n_features))
init['b2'] = tf.random_normal(seed=13, shape=[1], dtype=tf.float32, mean=0.0, stddev=np.sqrt(6.0))


W1 = tf.Variable(init['W1'])
W2 = tf.Variable(init['W2'])
b1 = tf.Variable(init['b1'])
b2 = tf.Variable(init['b2'])




A = tf.sigmoid(tf.matmul(X, W1) + b1)
logits = tf.matmul(A, W2) + b2


loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits, y))
train = tf.train.AdamOptimizer(1e-3).minimize(loss)



In [57]:
from sklearn.model_selection import StratifiedKFold

batches = [idx for _, idx in StratifiedKFold(n_splits=100, random_state=42).split(data['X'], data['y'])]
n_epochs = 500

with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for _ in xrange(n_epochs):
        for idx in batches:
            loss_ = s.run([loss, train], feed_dict={X: data['X'].iloc[idx], y: data['y'].iloc[idx].to_frame()})[0]
        print '\r', loss_,


0.687927
